In [10]:
import pandas as pd
import pyodbc
import openpyxl
from datetime import datetime

# PARAMETROS
fecha_inicio = '2021-01-01'
fecha_fin = '2025-4-30'
c1 = 'Classic'
c2 = 'Gold'


In [11]:
# DICCIONARIO XC Y SEGMENTO
segmentos = {
    c1: {1: 8000, 2: 15000},
    c2: {1: 20000, 2: 30000}
}

bandas = {
    c1: [300000],
    c2: [400000]
}

def asignar_xc(categoria, valor):
    if categoria == c1:
        if valor < bandas[c1][0]: return segmentos[c1][1]
        else: return segmentos[c1][2]
    elif categoria == c2:
        if valor < bandas[c2][0]: return segmentos[c2][1]
        else: return segmentos[c2][2]

def asignar_segmento(categoria, valor):
    if categoria == c1:
        if valor < bandas[c1][0]: return f'{c1} 1'
        else: return f'{c1} 2'
    elif categoria == c2:
        if valor < bandas[c2][0]: return f'{c2} 1'
        else: return f'{c2} 2'

#En caso de tener 3 bandas y 4 segmentos de asignación

# # DICCIONARIO XC Y SEGMENTO
# segmentos = {
#     c1: {1: 7000, 2: 8000, 3: 9000, 4: 12000},
#     c2: {1: 9000, 2: 12000, 3: 14000}
# }

# bandas = {
#     c1: [60000, 100000, 200000],
#     c2: [40000, 300000]
# }

# def asignar_xc(categoria, valor):
#     if categoria == c1:
#         if valor <= bandas[c1][0]: return segmentos[c1][1]
#         elif valor < bandas[c1][1]: return segmentos[c1][2]
#         elif valor <= bandas[c1][2]: return segmentos[c1][3]
#         else: return segmentos[c1][4]
#     elif categoria == c2:
#         if valor <= bandas[c2][0]: return segmentos[c2][1]
#         elif valor < bandas[c2][1]: return segmentos[c2][2]
#         else: return segmentos[c2][3]

# def asignar_segmento(categoria, valor):
#     if categoria == c1:
#         if valor <= bandas[c1][0]: return f'{c1}1'
#         elif valor < bandas[c1][1]: return f'{c1}2'
#         elif valor <= bandas[c1][2]: return f'{c1}3'
#         else: return f'{c1}4'
#     elif categoria == c2:
#         if valor <= bandas[c2][0]: return f'{c2}1'
#         elif valor < bandas[c2][1]: return f'{c2}2'
#         else: return f'{c2}3'




In [12]:
# CONEXION Y CONSULTAS
conn = pyodbc.connect(DRIVER='{SQL SERVER}', SERVER='PALDBBI01', DATABASE='PROD_dW')

# Consulta SQL PARA DEFLACTAR MAS ADELANTE
query = f"""
select a.playerid,
       max(b.categoria) AS categoria,
       concat(year(a.fechacontable), '-', month(a.fechacontable)) as fecha,
       sum(a.coininpesos) as coinin
from prod_Dw.dw.vwfactsesionjuego a WITH (NOLOCK)
join prod_Dw.dw.vwdimjugador b WITH (NOLOCK) on a.playerid=b.playerid
where a.playerid not in (
    select distinct c.playerid
    from prod_Dw.dw.vwfactsesionjuego c WITH (NOLOCK)
    where c.fechacontable >= '{fecha_fin}'
)
and b.vigente=1
and b.status='Active'
and b.categoria in ('{c1}','{c2}')
group by a.playerid, year(a.fechacontable), month(a.fechacontable)
order by a.playerid, year(a.fechacontable), month(a.fechacontable)
"""


base = pd.read_sql(query, conn)


#base.to_excel("C:/Users/bfrossasco/Desktop/Deflactado.xlsx" , index=False)


C:\Users\bfrossasco\AppData\Local\Temp\ipykernel_32472\3610633530.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  base = pd.read_sql(query, conn)


In [13]:
#calculo y merge

# Cargar archivo de inflación 
inflacion = pd.read_excel("C:\\Users\\bfrossasco\\Desktop\\Recupero Prueba\\Ian archivos\\inflacion actualizada 04-06.xlsx")

# Parsear la fecha en 'base' y crear columna 'inicio mes'
base['fecha'] = pd.to_datetime(base['fecha'], errors='coerce')
inflacion ['fecha'] = pd.to_datetime(inflacion['fecha'], errors='coerce')

# Merge con base
base2=pd.merge(base,inflacion, on='fecha', how='left')

#Cálculo
base2['NewCI']=base2['coinin']*base2['deflacion']

#Eliminación de columnas que no se utilizan
base2.drop(columns=['fecha','coinin','año','mes','inflacion','adicion','deflacion'], inplace=True)

# Base deflactada armada
base3=base2.groupby('playerid').agg({'categoria':'max','NewCI':'mean'}).reset_index()

#base3.to_excel("C:/Users/bfrossasco/Desktop/calculo.xlsx" , index=False) 


In [14]:
#ASIGNACION DE SEGMENTO Y XC

base3['xc'] = base3.apply(lambda row: asignar_xc(row['categoria'], row['NewCI']), axis=1)
base3['segmento'] = base3.apply(lambda row: asignar_segmento(row['categoria'], row['NewCI']), axis=1)

#base3.to_excel("C:/Users/bfrossasco/Desktop/asignacion.xlsx" , index=False) 

In [15]:
#SUBIR Y UNIR CLIENTES QUE NO JUGARON 2024 Y 2023

# Consulta SQL PARA DEFLACTAR MAS ADELANTE
query2 = f"""
declare @classic1 int
declare @gold1 int
set @classic1 = 8000
set @gold1= 20000


select distinct
playerid,
categoria,
case
when Categoria='Classic' then 'Classic 1'
when Categoria='Gold' then 'Gold 1'
end as segmento,
case
when Categoria='Classic' then @classic1
when Categoria='Gold' then @gold1
end as xc,
NewCI = 0
from prod_Dw.dw.vwdimjugador
where status ='Active' 
and vigente= 1 
and categoria in ('Classic','Gold')
and playerid not in (select distinct a.playerid from prod_dw.dw.vwfactsesionjuego a)
and playerid not in (select distinct a.playerid from prod_Dw.dw.VwFactSesionJuegoHist a)
UNION
select distinct 
a.playerid,
b.categoria,
case
when B.Categoria='Classic' then 'Classic 1'
when B.Categoria='Gold' then 'Gold 1'
end as segmento,
case
when B.Categoria='Classic' then @classic1
when B.Categoria='Gold' then @gold1
end as xc,
NewCI = 0
from prod_Dw.dw.VwFactSesionJuegoHist  a WITH (NOLOCK)
join prod_Dw.dw.VwDimJugador b WITH (NOLOCK)
on a.playerid=b.playerid
where a.playerid not in (select distinct c.playerid from prod_Dw.dw.vwfactsesionjuego c WITH (NOLOCK))
and b.categoria in ('Classic','Gold') 
and b.Vigente='1' 
and b.Status='Active'
"""

basehistorica = pd.read_sql(query2, conn)

#CONCATENAR CON LA TABLA YA DEFLACTADA
tabla_unida = pd.concat([base3, basehistorica], ignore_index=True)


#tabla_unida.to_excel("C:/Users/bfrossasco/Desktop/union1.xlsx" , index=False)



C:\Users\bfrossasco\AppData\Local\Temp\ipykernel_32472\251328151.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  basehistorica = pd.read_sql(query2, conn)


In [16]:
#Cruzar con SalesForce (SQL)
#sf = pd.read_excel("C:\\Users\\bfrossasco\\Desktop\\Recupero Py 2025\\Archivos\\Archivos SF\\SF 03-6-25.xlsx")

#Cruzar con data SF (SQL)
querysf = f"""select playerid,Nombre,CodCuenta,CodContacto,email,Celular,Documento 
from VwDimClienteSF
where PlayerId is not null""" 

sf = pd.read_sql(querysf, conn)

#Cruzar con archivo AAyPP
AAyPP=pd.read_excel("C:\\Users\\bfrossasco\\Desktop\\Recupero Py 2025\\Archivos\\Prohibidos y Exclusiones\\PROHIBIDOS 02-06.xlsx")



# Merge con base sin sacar excluidos y prohibidos
recupero_sin_AAyPP=pd.merge(tabla_unida,sf, on='playerid', how='left', suffixes=('_base', '_sf'))

#recupero_sin_AAyPP.to_excel("C:/Users/bfrossasco/Desktop/recupero_sin_aapp.xlsx" , index=False)

#borrar los que no tienen datos de SF y los que no tienen email
recupero_sin_AAyPP = recupero_sin_AAyPP[recupero_sin_AAyPP['CodCuenta'].notna()]
recupero_sin_AAyPP = recupero_sin_AAyPP[(recupero_sin_AAyPP['email'].notna()) & 
                                        (recupero_sin_AAyPP['email'] != '0')  & 
                                        (recupero_sin_AAyPP['email'].str.strip() != '')]

#Cambiar nombre de las columnas
recupero_sin_AAyPP = recupero_sin_AAyPP.rename(columns={ 'CodContacto':'Marketing Cloud ID',                                                        
                                                        'CodCuenta':'id de 18 caracteres'})                                                    
                                                        

#Agrega columna 'apellido' = 'nombre'
recupero_sin_AAyPP = recupero_sin_AAyPP.assign(Apellido=recupero_sin_AAyPP['Nombre'])

#Agrega columna TIPO_DNI
recupero_sin_AAyPP['Tipo de documento'] = 'DNI'

# #Borrar columnas adicionales (11 columnas quedan)
recupero_sin_AAyPP.drop(columns=['NewCI'],inplace=True)
recupero_sin_AAyPP.drop(columns=['categoria'],inplace=True)

# #Convertir a INT las columnas de Documento y Celular
# recupero_sin_AAyPP['Documento'] = pd.to_numeric(recupero_sin_AAyPP['Documento'])
# recupero_sin_AAyPP['Celular'] = pd.to_numeric(recupero_sin_AAyPP['Celular'])

#columnas ordenadas
columnas_ordenadas = ['id de 18 caracteres','Marketing Cloud ID','playerid','Nombre','Apellido','Tipo de documento','Documento','email','xc','segmento','Celular']

recupero_sin_AAyPP= recupero_sin_AAyPP[columnas_ordenadas]

# Merge con base y AAyPP
recupero_final=pd.merge(recupero_sin_AAyPP,AAyPP, on='Documento', how='left', suffixes=('_base', '_sf'))

# Eliminar los que sí hicieron match (es decir, los que están en AAyPP)
recupero_final = recupero_final[~recupero_final['Documento'].isin(AAyPP['Documento'])]


#Emails mas repetidos en esta lista
lista_emails =  [
    "noposee@gmail.com", "noposee@hotmail.com", "noposee@palermo.com.ar", "no@posee.com",
    "no@gmail.com", "noposee@noposee.com", "nopose@gmail.com", "no@hotmail.com",
    "noposee@msn.com", "n@gmail.com", "notiene@gmail.com", "noposeemail@palermo.com.ar",
    "nose@gmail.com", "noposee@live.com", "no@msn.com", "nopo@gmail.com",
    "nopose@hotmail.com", "notiene@hotmail.com", "noposee@outlook.com", "no@posee.com.ar",
    "noposeee@gmail.com", "n@hotmail.com", "noposee@hotmail.com.ar", "nopo@hotmail.com",
    "nooposee@gmail.com", "noposee1@gmail.com", "noposee@palermo.com", "wfg@gmail.com",
    "noposee@gmail.com.ar", "111@gmail.com", "noposee@yahoo.com", "no@live.com",
    "noposee@personal.com", "nopose@msn.com", "sinnombre@gmail.com", "noposeemail@gmail.com",
    "111111111111@gmail.com", "noposse@gmail.com", "sinespecificar@gmail.com", "a@hotmail.com",
    "nooposee@hotmail.com", "noposee@yahoo.com.ar", "n@msn.com", "nopo@msn.com",
    "noposeee@hotmail.com", "nopose@live.com", "nose@hotmail.com", "no@outlook.com",
    "noposee1@live.com", "k@hotmail.com", "1@gmail.com", "nop@gmail.com", "n@live.com",
    "mmm@hotmail.com", "nooposee@msn.com", "notienemail@gmail.com", "noposse@hotmail.com",
    "nooposee@live.com", "nopsee@gmail.com", "nopose@outlook.com", "nopo@live.com",
    "noposee@noposee.com.ar", "noposee@gmial.com", "j@hotmail.com", "yllbjlkbvjkljh@gmail.com",
    "nose@live.com", "nose@msn.com", "noposeemail@hotmail.com", "noposee1@hotmail.com",
    "norecuerda@gmail.com", "nopo@outlook.com", "nopoee@gmail.com", "ggg@hotmail.com",
    "@hotmail.com","{@gmail.com","0@live.com","0@msn.com","000@hotmail.com",
    "0@gmail.com","0@hotmail.com","111@hotmail.com","edf@gmail.com","no@yahoo.com","ni@gmail.com",
    "nooposee@outlook.com","nop@hotmail.com","nopoosee@palermo.com.ar","n.n@hotmail.com","no@no.com",
    "2@hotmail.com","k@gmail.com","c@yahoo.com","2@hotmail.com","sjdjsksk@gmail.com","sjdjsksk@msn.com",
    "nose@yahoo.com","nopos@gmail.com","noposeee@msn.com","noposee@gamil.com","n@outlook.com",
    "nopesee@gmail.com","ni@hotmail.com","nopose@yahoo.com","nn@gmail.com","noposee@posee.com",
    "nopossee@gmail.com","noinforma@gmail.com","nor@gmail.com","j@msn.com","j@gmail.com",
    "c@hotmail.com","0@msn.com","abcde@gmail.com","a@msn.com","2@gmail.com","11111@gmail.com",
    "a@gmail.com","yllbjlkbvjkljh@hotmail.com","t@msn.com","sjdjsksk@hotmail.com","notiene@gmail.com.ar",
    "nosabe@gmail.com", "nosep@gmail.com", "notiene/@gmail.com", "nopose@noposee.com", "noposse@hotmail.com.ar", 
    "nadie123@hotmail.com","no@posse.com", "n_p@hotmail.com", "noposeee@noposee.com", "nopose@gmail.com.ar", "noposee@gmai.com", "nop@msn.com","noposeeee@hotmail.com", "n0@gmail.com", "nopoesee@gmail.com", "noo@gmail.com", "l@msn.com", "l@gmail.com",
    "l@hotmail.com", "h@msn.com", "ha@gmail.com", "h@gmail.com", "j@live.com", "gg@live.com",
    "g@gmail.com", "desde@hotmail.com", "c@gmail.com", "11@gmail.com", "0@hotmail.com", "u@hotmail.com",
    "v@gmail.com", "yllbjlkbvjkljh@msn.com", "wfg@msn.com", "wfg@live.com", "xxx@gmail.com", "t@live.com",
    "s@hotmail.com", "npposee@gmail.com", "npposee@live.com", "notiene@msn.com", "o@hotmail.com", "nose@outlook.com",
    "notengo@gmail.com", "oooo@outlook.com", "noseacuerda@gmail.com", "noseee@gmail.com", "noseee@hotmail.com", "p@msn.com",
    "prueba@gmail.com", "q@hotmail.com", "noposee1@msn.com", "nopoee@hotmail.com", "noposseo@gmail.com", "noopo@msn.com",
    "noposeeee@outlook.com", "noposee@hotamail.com", "noposeve@palermo.com.ar", "noo@hotmail.com", "nooposee@yahoo.com", "noposee@nopsee.com","noposee3457@palermo.com.ar", "noposeeee@gmail.com", "noposeeeen@gmail.com", "noposse@yahoo.com.ar", "nopse@gmail.com", "nooooposee@gmail.com","noosee@hotmail.com", "nopoee@live.com", "nopos@hotmail.com", "noposee@gmaill.com", "noposee@mail.com", "noposee2@gmail.com","noposeeee@yahoo.com", "nopposee@gmail.com", "m@gmail.com", "jjjj@hotmail.com", "k@live.com", "f@msn.com","e@hotmail.com", "c@live.com","'@hotmail.com"
]



#Eliminar lista de emails no validos
recupero_final = recupero_final[~recupero_final['email'].isin(lista_emails)]


#Eliminar Unnamed: 0 
recupero_final.drop(columns=['Unnamed: 0'], inplace=True)


#eliminar por patrones
## Patrón general con expresiones regulares
#patron_invalidos = r'^(n|no|noposee|notiene|nose|nooposee|noposse|sinnombre|sinespecificar)[\w\d]*@'

# Filtro: eliminar correos que cumplan con ese patrón (case insensitive)
#df = df[~df['mail'].str.lower().str.match(patron_invalidos, na=False)]

#prueba
recupero_final.to_excel("C:/Users/bfrossasco/Desktop/Recupero 13-06-25.xlsx" , index=False)


C:\Users\bfrossasco\AppData\Local\Temp\ipykernel_32472\856136833.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sf = pd.read_sql(querysf, conn)
